# Assignment 4

## Data mining questions

1- Is fully-connected model a good one for sequential data? Why? How about for image data? Is it good? Why?

- No, It's not good, because the model is stateless, also since the sequential data is dependent on each other, but the neural network assumes that the data is non-sequential, it isloate features from each other and start training each feature alone, also it's not good for image preprocessing and classification, because it's not good at image feature extraction, and there will be a lot of learnable parameters and that may cause overfitting.

<hr>

2- What is gradient vanishing and gradient explosion, and how GRU/LSTM tries to mitigate this problem?

- They are unstable behaviourx happen while model goes backward to learn, gradient vanishing, the gradient get smaller and smaller get smaller and smaller untill it reach to zero, on other hand, gradient explosion, the gradient get larger and larger and this will make weights updated to worset and worset values.<br><br>LSTM has three gates (forget gate, input gate, output gate [outputs a prediction vector h(x) at k-th time step]), As a result, LSTM features a distinct gradient structure that allows direct access to the activations of the forget gate, allowing the network to encourage desired behaviour from the error gradient via frequent gate updates at each time step of the learning process, and then throw cell state **(∂s<sub>t</sub>'∂s<sub>t</sub>=∏<sub>t</sub>1t'-tσ(v<sub>t</sub>+k))** ,*v<sub>t</sub>* the output of forget state but still LSTM will suffer from vanishing gradients as well, also LSTM's activation function has the derivative of 1 so it won't neither vanish or explode.<br><br>GRU also has similar sturcture of LSTM but with only 2 Gates (reset and update gate) it handel vanishing and exploding gradient a little bit better than LSTM but it may also suffer from this problem.

<hr>

3- What is multi-objective/multi-task learning? What is multi-modality learning? How do you use them in this assignment?

- **Multi-task learning (MTL)** is a machine learning discipline in which numerous learning problems are handled concurrently, in other hand **Multi-Modality** learning makes use of data from multiple sources to produce intelligent predictions, in this assignment we have a special dataset where we have two inputs *(summary and image of some real estates)* and *(type and categorical price of this real estates)* I leveraged from these types of learning and I built a model and trained it on these inputs at the same time, and I could predict the values of type and price of the state.

<hr>

4- What is the difference among xgboost, lightgbm and catboost?

- All of these algorithms lies under gradient boosting algorithm. They differ in their implementation of the boosted trees algorithm, as well as their technological compatibility and limits, Trees grow depth-wise in XGBoost, while trees grow leaf-wise in LightGBM, for substitution,<br><br> 
 - lightGBM supports gradient-based one-sided sampling (GOSS), which selects the split by combining all instances with big gradients (i.e., large errors) and a random sample of examples with small gradients. To maintain the same data distribution when calculating the information gain.<br><br> 
 - Catboost offers a new technique called Minimal Variance Sampling (MVS), which is a weighted sampling version of Stochastic Gradient Boosting. Catboost provides the Minimal Variance Sample (MVS) technique, which is a weighted sampling variation of Stochastic Gradient Boosting,<br><br> 
 - Xgboosting doesn't introduce any weighted sampling techniques..




## Problem formulation

When hosts prepare to post a new advertisement about residence place on the Airbnb site is, how much should they ask for?, How much of this apartment?, etc, also for guests, they want to find a suitable place to stay at it affordable price..., So we want to build a neural network model to make predict the type of real estate and its price, our inputs here are summary for this real estate and its image and the outputs here are a type of this real estate and its price. *(training dataset:- *7627 samples)*, *(test dataset:- 7360 samples).

#### Note:- **you will find the dataset at https://www.kaggle.com/competitions/cisc-873-dm-f22-a4**

### Challengs:

- The size of training dataset is approximately equal to test dataset and it will be smaller when we drop null and duplicated values from training dataset.
- Null values in summary feature in training dataset.
- summary feature in both training dataset and test dataset is multilingual.
- imbalanced labels distribution.
- multi-prediction problem.
- huge time taken to train with cpu.
<br><hr>

### Impact:

- It will help hosts to put the true and suitable price for their residence place and that will increase their sales, also it will help guests to predict the night price for the place that they are looking for it to stay in it at holidays or in the work trip days.
<br><hr>

### Data mining function:

1. load the data from the source.
2. preprocess and manipulating the data.
3. build and train the model.
4. classification and prediction.
5. get insights from the results.
<br><hr>

### Ideal solution: 

For me using a bert model *(pre-trained model)* was a great choice, it got the highest score for me on kaggle, and I think that's because a lot of learnable parameters.

Actually this model is the best model that you can use it for this kind of problems, bert model is a great semantic and that because a lot of learnable parameters, more than 108 million trainable parameters. kaggle score (68.546), and I could got better result if I tried to tune the hyperparameters of this model.

## Experimental protocol

1. load train and test data.
2. some visualization and description to understand the data.
2. preprocessing the data
    1. preprocessing the data to build models from scratch.
        1. translate text data. (summary feature).
        2. remove null and duplicated values from summary feature in training dataset.
        3. translate summary feature (because it's multilingual data).
        4. resize images into (64\*64\*3) shape (to preserve more features as we could).
        5. tokenize each text data in summary feature.
        6. convert each text into unique id sequence.
    2. preprocessing the data to build a pre-trained model.
        1. translate text data. (summary feature).
        2. remove null and duplicated values from summary feature in training dataset.
        3. translate summary feature (because it's multilingual data).
        4. tokenize each text data in summary feature with pre-trained tokenizer.
        5. convert each text into unique id sequence.
3. build models and plot them.
4. train each model.
5. plot each model on training loss and validation loss.
6. plot each model on training accuracy and validation accuracy.

### REF:

- https://www.tensorflow.org/text/tutorials/classify_text_with_bert
- https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21
- https://neptune.ai/blog/when-to-choose-catboost-over-xgboost-or-lightgbm
- https://huggingface.co/docs/transformers/model_doc/bert
- 873_mml_mtl lab



## Import required packages

In [ ]:
import pandas as pd

import os
import pathlib

import numpy as np
import matplotlib.pyplot as plt
from time import time
from tqdm.notebook import tqdm

#to translate the text data
!pip install googletrans
!pip install googletrans==3.1.0a0 --user
from googletrans import Translator
import cv2


#bert tokenizer and bert model
from transformers import BertTokenizer
from transformers import TFBertModel

#deep learning packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, LSTM, Dropout, Bidirectional, GRU
from tensorflow.keras.optimizers import Adam, Nadam
from keras.models import Model

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pprint import pprint

#nlp

from nltk.stem import WordNetLemmatizer
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')


## Load the data from the source

In [2]:
train_df = pd.read_csv("../input/airbnb/train_xy.csv")
test_df = pd.read_csv("../input/airbnb/test_x.csv")

In [3]:
#describe object features in training data
train_df[['summary','image','type']].describe()

In [4]:
#describe train data (price values)
train_df.describe()

In [5]:
#decribe test data
test_df.describe()

In [6]:
print("Some information about training dataset.\n")
print(train_df.info())
print("\n============================================\n")
print("Some information about test dataset.\n")
print(test_df.info())

## preprocessing the data

### remove null values from training dataset

In [7]:
print(f"number of rows in training dataset after dropping all null values {train_df.shape[0]} value")
train_df = train_df.dropna()
print(train_df.shape)

print("Some information about training dataset.\n")
print(train_df.info())

### check the number of duplicated rows in summary feature and remove them.

In [8]:
#how many duplicated rows in summary feature
print(f"number of duplicated rows in summary feature in training dataset are {train_df.duplicated(subset='summary',keep='first').sum()} values.")
train_df = train_df.drop_duplicates(subset='summary',keep='first')
train_df.reset_index(drop=True, inplace=True)
print(f"number of rows in summary feature in training dataset after dropping duplicated rows is {train_df.shape[0]} value.")

In [9]:
#check now if how many null values in both datasets
print(f"number of null values in training dataset is {train_df.isnull().sum().sum()} value.")
print(f"number of null values in testing dataset is {test_df.isnull().sum().sum()} value.")
#check now if how many duplicated values in both datasets
print(f"number of duplicated rows in summary feature in training dataset is {train_df.duplicated(subset='summary',keep='first').sum()} value.")
#there are still duplicated rows in summary feature in testing dataset but we can't remove them.
print(f"number of duplicated rows in summary feature in testing dataset are {test_df.duplicated(subset='summary',keep='first').sum()} values.")

### translate training summary feature and testing summary feature. 

In [10]:
start_time = time()

#translate the summary column for both datasets
print('Start translating the training dataset')
for i in tqdm(range(len(train_df))):
    translator = Translator()
    train_df.at[i,'summary'] = translator.translate(train_df.at[i,'summary'] , dest='en').text
    
print("Done..\n")
    
print('Start translating testing dataset')
for i in tqdm(range(len(test_df))):
    translator = Translator()
    test_df.at[i,'summary'] = translator.translate(test_df.at[i,'summary'] , dest='en').text
print("Done..\n")



print(len(train_df['summary']))
print(len(train_df['image']))
print(len(train_df['type']))
print(len(train_df['price']))
test_df['image']

end_time = time()
total_time = end_time - start_time
result = '{0:02.0f} minutes and {1:02.0f} seconds'.format(*divmod((total_time/60) * 60, 60))
print(f"The total time taken to translate summary feature in both datasets was: {result}")


### Some data visualization

In [11]:
train_df['type'].value_counts().sort_values(ascending = False).plot(kind='bar')

In [12]:
train_df['price'].value_counts().sort_values(ascending = False).plot(kind='bar')

### split training data into training and validation set.

### Images data preprocessing
Resize each image in image feature and store its value in numpy array (do that for both, training set and test set).

In [13]:
start_time = time()

# preprocess image data
def load_image(file):
    p = pathlib.Path(file)
    dir_and_file_name = os.path.join(*p.parts[-2:])
    #if you're using kaggle notebook to run this code unhash following line.
    file = f"../input/airbnb/{dir_and_file_name}"
    try:
        #read image first
        image = cv2.imread(file, 1)
        #resize it
        image = cv2.resize(image, (64,64), interpolation=cv2.INTER_CUBIC) #read it as rgb to preserve more information about each image.
        #store its value
        arr = np.array(image)
    except:
        #if the file doesn't exsit store array of zeros
        arr = np.zeros((64, 64, 3))
    return arr


# loading images:
x_train_image = np.array([load_image(i) for i in tqdm(train_df.image)])
x_test_image = np.array([load_image(i) for i in tqdm(test_df.image)])

# change summary type: 
train_df.summary = train_df.summary.astype('str')
test_df.summary = test_df.summary.astype('str')

# get type 
y_train_type = train_df.type

# get price
y_train_price = train_df.price
end_time = time()
total_time = end_time - start_time
result = '{0:02.0f} minutes and {1:02.0f} seconds'.format(*divmod((total_time/60) * 60, 60))

print(f"The total time taken to load images from both datasets was: {result}")

In [14]:
#check if training images loaded correctly
print(x_train_image[0])

In [15]:
#check if test images loaded correctly
x_test_image[0]

In [16]:
print(x_train_image.shape)
print(x_test_image.shape)
print(train_df.summary.shape)
print(test_df.summary.shape)

### text data preprocessing

tokenize each text in summary feature and then give each token a unique id based on keras tokenizer.

In [17]:
vocab_size = 50000
max_len = 100


# build vocabulary from training set
mm_tokenizer = Tokenizer(num_words=vocab_size)
mm_tokenizer.fit_on_texts(train_df.summary)
mm_tokenizer.fit_on_texts(test_df.summary)


def _preprocess(list_of_text):
    return pad_sequences(
        mm_tokenizer.texts_to_sequences(list_of_text),
        maxlen=max_len,
        padding='post',
    )
    

# padding is done inside: 
x_train_text_id = _preprocess(train_df.summary)
x_test_text_id = _preprocess(test_df.summary)

print(x_train_text_id.shape)
print(x_test_text_id.shape)

In [18]:
# we can use the tokenizer to convert IDs to words.
pprint(mm_tokenizer.sequences_to_texts(x_train_text_id[:5]))

### text data preprocessing for pretrained model.

In [19]:
## define pretrained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [20]:
#check how the value will be after encode it to unique ids
token = tokenizer.encode_plus(
    train_df['summary'].iloc[611],
    max_length=190, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)
token

In [21]:
input_ids = np.zeros((len(train_df['summary']), 200))
masks = np.zeros((len(train_df['summary']), 200))

### generate_data function

- Tokenizing (splitting strings in sub-word token strings), converting tokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).
- Adding new tokens to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).
- Managing special tokens (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.

In [22]:
def generate_data(df, tokenizer,  ids, masks):
    for i, text in tqdm(enumerate(df)):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=200,  
            padding='max_length',
            truncation=True,
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [23]:
input_ids, masks = generate_data(train_df['summary'], tokenizer, input_ids, masks)

In [24]:
labels = np.zeros((len(train_df['price']), 3))

print(labels.shape)
train_df['price']

In [25]:
#convert labels values to one hot encoded data.
labels[np.arange(len(train_df['price'])), train_df['price'].values] = 1 # one-hot encoded the label tensor

In [26]:
labels

In [27]:
#create tensor dataset
dataset = tf.data.Dataset.from_tensor_slices((input_ids, masks, labels))

In [28]:
#function to make a tensor dataset like dataframe with 3 features (2 inputs [ids, mask]) and one output(one hot encoded labels)
def dataset_map(input_ids, attention_mask, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }, labels

In [29]:
dataset = dataset.map(dataset_map) # converting to required format for tensorflow dataset

In [30]:
dataset = dataset.shuffle(10000).batch(32, drop_remainder=True) # batch size, drop any left out tensor

In [31]:
batch_train_size = 0.8
train_size = int((len(train_df['summary'])//32)*batch_train_size) # for each 32 batch of data we will have len(df)//16 samples, take 80% of that for train.

In [32]:
#creat training dataset
training_dataset = dataset.take(train_size)
validation_dataset = dataset.skip(train_size)
print(type(training_dataset))

In [33]:
def plot_performance(history, model_number, training_metrics, validation_metrics, target):
    """
    function to plot training price loss vs validation price loss and training price accuracy vs validation price accuracy.<br>
    
    params:
    
    history: model.fit object
    model number (int): just integer number to indicate the model number.
    training_metrics (list) (2 values): first values like 'loss' will passed to history object
    validation_metrics (list) (2 values): first values like 'val_loss' will passed to history object
    target (string): just a string to diplay it on the plot set it either price or type.
    """
    val_loss_per_epoch = history.history[validation_metrics[0]]
    loss_per_epoch = history.history[training_metrics[0]]
    val_accuracy_per_epoch = history.history[validation_metrics[1]]
    accuracy_per_epoch = history.history[training_metrics[1]]
    plt.figure(figsize=(8,8))
    plt.title(f"model number {model_number} {target} training loos & {target} validation loss with batch size 64")
    plt.xlabel('epoch')
    plt.ylabel('loss function')
    plt.plot(np.arange(1,len(val_loss_per_epoch)+1),val_loss_per_epoch,label=f"validation loss")
    plt.plot(np.arange(1,len(loss_per_epoch)+1),loss_per_epoch,label = f"training loss")
    plt.legend(loc="upper left")
    plt.show()
    plt.figure(figsize=(8,8))
    plt.title(f"model number {model_number} {target} training accuracy & {target} validation accuracy with batch size 64")
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.plot(np.arange(1,len(val_accuracy_per_epoch)+1),val_accuracy_per_epoch,label="validation accuracy")
    plt.plot(np.arange(1,len(accuracy_per_epoch)+1),accuracy_per_epoch,label = "training accuracy")
    plt.legend(loc="upper left")
    plt.show()


### First trail:

Here, I used a simple **multi-modality** and **multi-Task** model like in the lab to predict the type and the price of the real estate, the model has embedded layer to handel text data and one convolution layer and max-pooling layer to handel image data, then 2 fully connected neural network in parallel to make the model learn more and to see if the model will learn if the number of learnable parameters increased or it'll overfit. *there's a dropout layer between each fully connected layer*, the optimizer here will be **nesterov adam** because this **optimizer** have nesterov momentum unlike adam optimizer which use momentum, so it'll converge faster than adam optimizer.<br>

**5,246,459 learnable parameters**

In [34]:
#first output shape
len_type = len(train_df.type.unique())
#second output shape
len_price = len(train_df.price.unique())
train_df['type'] = train_df.type.astype('category').cat.codes

# here we have two inputs. one for image and the other for text.
in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 3))


#print(reshape.dtype)

keras.backend.clear_session()


# text part
# simple average of embedding.
embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
averaged = tf.reduce_mean(embedded, axis=1)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)

# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task

fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


fused2 = Dense(256, activation='relu')(fused)
dropout_2 = Dropout(rate = 0.3)(fused2)
fused2 =  Dense(128, activation='relu')(dropout_2)

p_type = Dense(len_type, activation='softmax', name='type')(fused1)
p_price = Dense(len_price, activation='softmax', name='price')(fused2)


# define model input/output using keys.
first_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'type': p_type,
        'price': p_price,
    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
first_model.compile(
    optimizer=Nadam(),
    loss={
        'type': 'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type': 0.5,
        'price': 0.5,       
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy'],
    },
)


first_model.summary()
#plot the model
tf.keras.utils.plot_model(first_model, show_shapes=True)

### Start training the first model

I'll train the model with 30 epochs and 64 batch size and 20% of the total data as validation data, also I passed early stopping callback to the model to make the model stop training if the price validation accuracy decreased for three times.

In [35]:
history_1 = first_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },

    y={
        'type': train_df.type,
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [36]:
#plot price performance for first model
plot_performance(history_1, 1, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for first model
plot_performance(history_1, 1, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using first model.

In [37]:
#start predicting the values of type and price using summary and image as inputs
first_prediction = first_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
first_prediction = first_prediction['price']
print(first_prediction)

# categories
first_prediction = np.argmax(first_prediction, axis=1)

### submission file for kaggle using first model.

In [38]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': first_prediction}
).to_csv('sample_submission_1.csv', index=False)

### first trial result:

Amoung all next models, this model beat models which more complex than this simple model, it got score on kaggle (0.62934), and I think because it's the simplest **multi-modality** and **multi-Task** model, and the model didn't overfit on the training dataset.

### Second Trial

Here I used a similar model to that in first trail with the same objective and more imporved structure but to imporve the accuracy of the model I added a LSTM layer to between the embedded layer and averaged layer,because in terms of memory. Having a good hold over memorizing certain patterns LSTMs and it is effective in memorizing important information.
so it will perform fairly better, also I added a convolution layer after the max-pooling layer to extract more features in images and one dropout layer with 0.2 rate between the first convolution layer to make the model more robust to overfitting, with same optimizer in the first trial.<br>

**5,380,219 trainable parameters**


In [39]:
keras.backend.clear_session()

# text part
# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
lstm = LSTM(units = 128,return_sequences=True)(embedded)
averaged = tf.reduce_mean(lstm, axis=1)
print(averaged)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (7, 7))(in_image)
dropout_cov = Dropout(rate=0.2)(cov)
pl = MaxPool2D((8, 8))(dropout_cov)
cov_2 = Conv2D(32, (5, 5))(pl)

flattened = Flatten()(cov_2)


# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task

fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


fused2 = Dense(256, activation='relu')(fused)
dropout_2 = Dropout(rate = 0.3)(fused2)
fused2 =  Dense(128, activation='relu')(dropout_2)

p_type = Dense(len_type, activation='softmax', name='type')(fused1)
p_price = Dense(len_price, activation='softmax', name='price')(fused2)


# define model input/output using keys.
second_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'type': p_type,
        'price': p_price,
    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
second_model.compile(
    optimizer=Nadam(),
    loss={
        'type': 'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type': 0.5,
        'price': 0.5,       
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy'],
    },
)


second_model.summary()
#plot the model
tf.keras.utils.plot_model(second_model, show_shapes=True)

## Start training the second model

same as the first trail.

In [40]:
history_2 = second_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },

    y={
        'type': train_df.type,
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [41]:
#plot price performance for second model
plot_performance(history_2, 2, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for second model
plot_performance(history_2, 2, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using second model.

In [42]:
second_prediction = second_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
second_prediction = second_prediction['price']
print(second_prediction)

# categories
second_prediction = np.argmax(second_prediction, axis=1)

### submission file for kaggle using second model.

In [43]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': second_prediction}
).to_csv('sample_submission_2.csv', index=False)

### second trial result:

this model is a little bit simillar to the previous one, it got a little bit lower score on kaggle (0.62826), and I think that's because I added LSTM layer and make the image classifier layer more complex, lstm layer here could handel vanishing gradient and gradient explosion problems.

### Third trial

Here I used the same model in the second trail with the same objective but to make a model more effecient in terms of computaion I replaced the LSTM layer with GRU layer between the embedded layer and averaged layer, I used it for the same reason that I used LSTM layer *(explained in the second trial)*, but GRU layer is more effecient in terms of computation and maybe give me a good accuracy since it's has two gates **(update gate and reset gate)**.<br>

**5,351,291 trainable parameters** 

In [44]:
keras.backend.clear_session()
# here we have two inputs. one for image and the other for text.

# text part
# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
gru = GRU(units = 128,return_sequences=True)(embedded)
averaged = tf.reduce_mean(gru, axis=1)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (7, 7))(in_image)
dropout_cov = Dropout(rate=0.2)(cov)
pl = MaxPool2D((8, 8))(dropout_cov)
cov_2 = Conv2D(32, (5, 5))(pl)

# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task

fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


fused2 = Dense(256, activation='relu')(fused)
dropout_2 = Dropout(rate = 0.3)(fused2)
fused2 =  Dense(128, activation='relu')(dropout_2)

p_type = Dense(len_type, activation='softmax', name='type')(fused1)
p_price = Dense(len_price, activation='softmax', name='price')(fused2)


# define model input/output using keys.
third_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'type': p_type,
        'price': p_price,
    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
third_model.compile(
    optimizer=Nadam(),
    loss={
        'type': 'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type': 0.5,
        'price': 0.5,       
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy'],
    },
)


third_model.summary()
#plot the model
tf.keras.utils.plot_model(third_model, show_shapes=True)

## Start training the third model

same as the first trail.

In [45]:
history_3 = third_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },

    y={
        'type': train_df.type,
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [46]:
#plot price performance for third model
plot_performance(history_3, 3, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for third model
plot_performance(history_3, 3, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using third model.

In [47]:
third_prediction = third_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
third_prediction = third_prediction['price']
print(third_prediction)

# categories
third_prediction = np.argmax(third_prediction, axis=1)

### submission file for kaggle using third model.

In [48]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': third_prediction}
).to_csv('sample_submission_3.csv', index=False)

### third trial result:

this model faild in predicting price values, it got score on kaggle (0.33423), I think GRU layer here didn't help the model and it suffered from gradient explosion.

## Fourh trail:

Same as second trial but making lstm layer bidirectional in order to improve the accuracy, bidirectional lstm will memorize the sequence backward (from the future to the past) and forward (from the past to the future), and I think that will imporve the accuracy of this model. <br>

**5,546,619 trainable parameters**

In [49]:
keras.backend.clear_session()

# text part
# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
bi_lstm = Bidirectional(LSTM(units = 128,return_sequences=True))(embedded)
averaged = tf.reduce_mean(bi_lstm, axis=1)



# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (7, 7))(in_image)
dropout_cov = Dropout(rate=0.2)(cov)
pl = MaxPool2D((8, 8))(dropout_cov)
cov_2 = Conv2D(32, (5, 5))(pl)

# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task
fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


fused2 = Dense(256, activation='relu')(fused)
dropout_2 = Dropout(rate = 0.3)(fused2)
fused2 =  Dense(128, activation='relu')(dropout_2)

p_type = Dense(len_type, activation='softmax', name='type')(fused1)
p_price = Dense(len_price, activation='softmax', name='price')(fused2)


# define model input/output using keys.
fourth_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'type': p_type,
        'price': p_price,
    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
fourth_model.compile(
    optimizer=Nadam(),
    loss={
        'type': 'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type': 0.5,
        'price': 0.5,       
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy'],
    },
)


fourth_model.summary()
#plot the model
tf.keras.utils.plot_model(fourth_model, show_shapes=True)

### Start training the fourth model

In [50]:
history_4 = fourth_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },

    y={
        'type': train_df.type,
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [51]:
#plot price performance for fourth model
plot_performance(history_4, 4, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for fourth model
plot_performance(history_4, 4, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using fourth model.

In [52]:
fourth_prediction = fourth_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
fourth_prediction = fourth_prediction['price']
print(fourth_prediction)

# categories
fourth_prediction = np.argmax(fourth_prediction, axis=1)

### submission file for kaggle using fourth model.

In [53]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': fourth_prediction}
).to_csv('sample_submission_4.csv', index=False)

### fourth trial result:

this model is the worest model amoung all trial, it got higher score on kaggle (0.30353), and I think because I took into considration the time flow because I made LSTM layer biderctional, and that didn't help the model, you can see from the plot that the validation accuracy kept decreased and that's maybe gradient explosion.

### first model result:

on kaggle it got lowset accuracy amoung these models and, I think becuase it's the simplest 

## Fifth trail

Same as the third trial but making **GRU** layer bidirectional in order to improve the accuracy, bidirectional gru will memorize the sequence backward (from the future to the past) and forward (from the past to the future), and I think that will imporve the accuracy of this model, and it'll be more efficient in terms of computation.

**5,483,003 trainable parameters**

In [54]:
keras.backend.clear_session()
# here we have two inputs. one for image and the other for text.


# text part
# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
bi_gru =  Bidirectional(GRU(units = 128,return_sequences=True))(embedded)
averaged = tf.reduce_mean(bi_gru, axis=1)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)

# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# multi-task learning (each is a multi-class classification)
# one dense layer for each task

fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


fused2 = Dense(256, activation='relu')(fused)
dropout_2 = Dropout(rate = 0.3)(fused2)
fused2 =  Dense(128, activation='relu')(dropout_2)

p_type = Dense(len_type, activation='softmax', name='type')(fused1)
p_price = Dense(len_price, activation='softmax', name='price')(fused2)


# define model input/output using keys.
fifth_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'type': p_type,
        'price': p_price,
    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
fifth_model.compile(
    optimizer=Nadam(),
    loss={
        'type': 'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type': 0.5,
        'price': 0.5,       
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy'],
    },
)


fifth_model.summary()
#plot the model
tf.keras.utils.plot_model(fifth_model, show_shapes=True)

### Start training the fifth model

In [55]:
history_5 = fifth_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },

    y={
        'type': train_df.type,
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [56]:
#plot price performance for fifth model
plot_performance(history_5, 5, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for fifth model
plot_performance(history_5, 5, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using fifth model.

In [57]:
fifth_prediction = fifth_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
fifth_prediction = fifth_prediction['price']
print(fifth_prediction)

# categories
fifth_prediction = np.argmax(fifth_prediction, axis=1)

### submission file for kaggle using fifth model.

In [58]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': fifth_prediction}
).to_csv('sample_submission_5.csv', index=False)

### fifth trial result:

this model is better than previous one in score, it got higher score on kaggle (0.62826), because I made GRU biderctional,which made the accuracy better.

## Sixth model

Here, I built a simple **multi-modality** and **single-task** model like fifth model to predict the price of the real estate, the model has the same structe as fifth model in terms of input and hidden layers but it has only one output since it's single-task model, and I built this model to see if it will perform same as the fifth model or the result will change. <br>

**5,304,803 trainable parameters**

In [59]:
keras.backend.clear_session()
# here we have two inputs. one for image and the other for text.
in_text = keras.Input(batch_shape=(None, max_len))
in_image = keras.Input(batch_shape=(None, 64, 64, 3))

#reshape = tf.keras.layers.Reshape((10, 10), input_shape=(None, max_len))(in_text)

#print(reshape.dtype)

keras.backend.clear_session()


# text part
# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
bi_gru =  Bidirectional(GRU(units = 128,return_sequences=True))(embedded)
averaged = tf.reduce_mean(bi_gru, axis=1)

# image part 
# simple conv2d. you can change it to anything else as needed
cov = Conv2D(32, (16, 16))(in_image)
pl = MaxPool2D((16, 16))(cov)
flattened = Flatten()(pl)

# fusion - combinig both
fused = tf.concat([averaged , flattened], axis=-1)

# single-task learning
# one dense layer for each task

fused1 = Dense(128, activation='relu')(fused)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)


p_price = Dense(len_price, activation='softmax', name='price')(fused1)


# define model input/output using keys.
sixth_model = keras.Model(
    inputs={
        'summary': in_text,
        'image': in_image
    },
    outputs={
        'price': p_price,
    },
)


# compile model with optimizer, loss values for task, loss 
# weights for the task.
sixth_model.compile(
    optimizer=Nadam(),
    loss={
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'price': 1,       
    },
    metrics={
        'price': ['SparseCategoricalAccuracy'],
    },
)


sixth_model.summary()
#plot the model
tf.keras.utils.plot_model(sixth_model, show_shapes=True)

### Start training the sixth model

In [60]:
history_6 = sixth_model.fit(
    x={
        'summary': x_train_text_id,
        'image': x_train_image
    },
    y={
        'price': train_df.price,
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [61]:
#plot price performance for sixth model
plot_performance(history_6, 6, ['loss','sparse_categorical_accuracy'], ['val_loss','val_sparse_categorical_accuracy'], 'price')

### start predicting testing price values using sixth model.

In [62]:
sixth_prediction = sixth_model.predict(
    {
        'summary': x_test_text_id,
        'image': x_test_image
    }
)


# probabilities
sixth_prediction = sixth_prediction['price']
print(sixth_prediction)

# categories
sixth_prediction = np.argmax(sixth_prediction, axis=1)

### submission file for kaggle using sixth model.

In [63]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': sixth_prediction}
).to_csv('sample_submission_6.csv', index=False)

### sixth trial result:

this model is built with bi-derctional GUR layer to memorize text sequence, it got score on kaggle (0.62934), and I noticed that the model stop learning at some point, but it's still better a little bit than previous model.

## Last trail:

I built a simple **single-modality** and **multi-task** model to predict the type and the price of real estate, it has an embedded layer -> bidrectional lstm layer to memorize the patterns in both directions -> averaged layer -> 2 fully connected layer to extract more features and imporve the learning process -> 2 output layers with softmax activation function one for type prediction and the other for price prediction. <br>

**5,307,355 trainable parameters**

In [64]:
keras.backend.clear_session()

# simple average of embedding. you can change it to anything else as needed

embedded = keras.layers.Embedding(mm_tokenizer.num_words, 100)(in_text)
bi_lstm =  Bidirectional(LSTM(units = 128,return_sequences=True))(embedded)
averaged = tf.reduce_mean(bi_lstm, axis=1)


# multi-task learning (each is a multi-class classification)
# one dense layer for each task

fused1 = Dense(128, activation='relu')(averaged)
dropout_1 = Dropout(rate = 0.2)(fused1)
fused1 = Dense(256, activation='relu')(dropout_1)

p_price = Dense(len_price, activation='softmax', name='price')(fused1)
p_type = Dense(len_type, activation='softmax', name='type')(fused1)


# define model input/output using keys.
seventh_model = keras.Model(
    inputs={
        'summary': in_text,
    },
    outputs={
        'type': p_type,
        'price': p_price,

    },
)


# compile model with optimizer, loss values for each task, loss 
# weights for each task.
seventh_model.compile(
    optimizer=Nadam(),
    loss={
        'type':'sparse_categorical_crossentropy',
        'price': 'sparse_categorical_crossentropy',
    },
    loss_weights={
        'type':0.5,
        'price': 0.5
    },
    metrics={
        'type': ['SparseCategoricalAccuracy'],
        'price': ['SparseCategoricalAccuracy']
    },
)


seventh_model.summary()
#plot the model
tf.keras.utils.plot_model(seventh_model, show_shapes=True)

### Start training the last model

In [65]:
history_7 = seventh_model.fit(
    x={
        'summary': x_train_text_id,
    },

    y={
        'type': train_df.type,
        'price': train_df.price
    },
    epochs=30,
    batch_size=64,
    validation_split = 0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_price_sparse_categorical_accuracy', patience=5, )],
    verbose=1
)

In [66]:
#plot price performance for last model
plot_performance(history_7, 7, ['price_loss','price_sparse_categorical_accuracy'], ['val_price_loss','val_price_sparse_categorical_accuracy'], 'price')
#plot type performance for last model
plot_performance(history_7, 7, ['type_loss','type_sparse_categorical_accuracy'], ['val_type_loss','val_type_sparse_categorical_accuracy'], 'type')

### start predicting testing price values using seventh model.

In [67]:
seventh_prediction = seventh_model.predict(
    {
        'summary': x_test_text_id,
    }
)


# probabilities
seventh_prediction = seventh_prediction['price']
print(seventh_prediction)

# categories
seventh_prediction = np.argmax(seventh_prediction, axis=1)


### submission file for kaggle using seventh model.

In [68]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': seventh_prediction}
).to_csv('sample_submission_7.csv', index=False)

### last trial result:

Actually this model got lower accuracy than **multi-modality and single-tasking** model and it is not good but still better than some trials here, because from one input it can't get good result for two outputs, kaggle score (0.61494).

## Pretrained model (Bert) - bouns section

Here, I decided to use a complex pretrained model and make it train on my text data only and predict only the price values, because bert model only learn on text data not images and then I added a hidden layer and the output layers with 3 units.<br>

**108,705,539 trainable parameters**

In [69]:
pretrained_model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

In [78]:
input_ids = tf.keras.layers.Input(shape=(200,), name='input_ids', dtype='int32')
attention_mask = tf.keras.layers.Input(shape=(200,), name='attention_mask', dtype='int32')

bert_embds = pretrained_model.bert(input_ids, attention_mask=attention_mask)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
hidden_layer = tf.keras.layers.Dense(320, activation='relu', name='hidden_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(3, activation='softmax', name='output_layer')(hidden_layer) # softmax activation function -> calculates probabilities of classes

bert_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)
bert_model.summary()
tf.keras.utils.plot_model(bert_model, show_shapes=True)

In [79]:
#learning rate with learning rate scheduler
opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-6)
#metrics is accuracy here because I encoded the price target to one-hot-encoded data.
bert_model.compile(optimizer=opt, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.CategoricalAccuracy('accuracy')])

In [80]:
tf.debugging.set_log_device_placement(True)
hist = bert_model.fit(
    training_dataset,
    validation_data=validation_dataset,
    epochs=25,
    workers=1500,
    use_multiprocessing=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
    ],
)

In [81]:
#plot price performance for last model (the hero)
plot_performance(hist, 1, ['loss','accuracy'], ['val_loss','val_accuracy'], 'price')

In [82]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

#preprocess testing dataset input as we did with training dataset input
def prepare_data(test_text, tokenizer):
    data = []
    for i, text in tqdm(enumerate(test_text)):
        token = tokenizer.encode_plus(
            test_text.iloc[i],
            max_length=200, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        data.append({
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
        })
        
    return data

#prediction function will predict each text observation using for loop
def make_prediction(model, processed_data,classes=[0,1,2]):
    prediction_data = []
    for i in tqdm(range(len(processed_data))):
        prediction =  model.predict(processed_data[i])[0]
        prediction_data.append(classes[np.argmax(prediction)])
    return prediction_data



processed_data = prepare_data(test_df['summary'], tokenizer)

prediction_data = make_prediction(bert_model, processed_data)

print(len(prediction_data))
print(prediction_data)

### submission file for kaggle

In [83]:
pd.DataFrame(
    {'id': test_df['id'],
     'price': prediction_data}).to_csv('sample_submission_pretrained_model.csv', index=False)